# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

list_elements = list(generator)

for sqrt_value in generator:
    print(sqrt_value)


In [2]:
sum(list_elements)

8.382332347441762

In [3]:
limit = 13
generator = square_root_generator(limit)
list_elements = list(generator)

for sqrt_value in generator:
    print(sqrt_value)

In [4]:
list_elements[-1]

3.605551275463989

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [6]:
data1 = list(people_1())
data1

[{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'},
 {'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'},
 {'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'},
 {'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'},
 {'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}]

In [7]:
data2 = list(people_2())
data2

[{'ID': 3,
  'Name': 'Person_3',
  'Age': 33,
  'City': 'City_B',
  'Occupation': 'Job_3'},
 {'ID': 4,
  'Name': 'Person_4',
  'Age': 34,
  'City': 'City_B',
  'Occupation': 'Job_4'},
 {'ID': 5,
  'Name': 'Person_5',
  'Age': 35,
  'City': 'City_B',
  'Occupation': 'Job_5'},
 {'ID': 6,
  'Name': 'Person_6',
  'Age': 36,
  'City': 'City_B',
  'Occupation': 'Job_6'},
 {'ID': 7,
  'Name': 'Person_7',
  'Age': 37,
  'City': 'City_B',
  'Occupation': 'Job_7'},
 {'ID': 8,
  'Name': 'Person_8',
  'Age': 38,
  'City': 'City_B',
  'Occupation': 'Job_8'}]

### First Iterator

Export data to DuckDB

In [8]:
import dlt
import duckdb

pipeline = dlt.pipeline(pipeline_name="HW1", destination="duckdb", dataset_name="hw")
info = pipeline.run(data1, table_name="users", write_disposition="merge", primary_key="ID")

Read data from DuckDB

In [9]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path= '{pipeline.dataset_name}'")
print(conn.sql("SHOW TABLES"))

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
└─────────────────────┘



In [10]:
users = conn.sql("SELECT * FROM users ORDER BY id").df()
display(users)

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708016329.5292494,Z/Dwetw3NG9j1w
1,2,Person_2,27,City_A,1708016329.5292494,hpQwi6RWIfm+0Q
2,3,Person_3,28,City_A,1708016329.5292494,ZahtW+wrhBGI6w
3,4,Person_4,29,City_A,1708016329.5292494,igEtsAaMSdyIWA
4,5,Person_5,30,City_A,1708016329.5292494,34hciO3g2alIPA


### Second Iterator

In [11]:
info = pipeline.run(data2, table_name="users", write_disposition="append", primary_key="ID")

In [12]:
users = conn.sql("SELECT * FROM users ORDER BY id").df()
display(users)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708016329.5292494,Z/Dwetw3NG9j1w,None
1,2,Person_2,27,City_A,1708016329.5292494,hpQwi6RWIfm+0Q,None
2,3,Person_3,28,City_A,1708016329.5292494,ZahtW+wrhBGI6w,None
3,3,Person_3,33,City_B,1708016331.1107492,NjnWhSapQ3uNhQ,Job_3
4,4,Person_4,29,City_A,1708016329.5292494,igEtsAaMSdyIWA,None
5,4,Person_4,34,City_B,1708016331.1107492,t3jShrLwsUwVDQ,Job_4
6,5,Person_5,30,City_A,1708016329.5292494,34hciO3g2alIPA,None
7,5,Person_5,35,City_B,1708016331.1107492,ygruBAwMNYhSAQ,Job_5
8,6,Person_6,36,City_B,1708016331.1107492,T4tvXIO5RiXjXw,Job_6
9,7,Person_7,37,City_B,1708016331.1107492,8zonwhkKeHXY3Q,Job_7


In [13]:
age_sum_1 = conn.sql("SELECT SUM(age) FROM users AS total_age").df()
display(age_sum_1)

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

### First Generator

In [14]:
pipeline = dlt.pipeline(pipeline_name="HW2", destination="duckdb", dataset_name="hw")
info = pipeline.run(data1, table_name="users", write_disposition="merge", primary_key="ID")

In [15]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path= '{pipeline.dataset_name}'")
print(conn.sql("SHOW TABLES"))

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
└─────────────────────┘



In [16]:
users = conn.sql("SELECT * FROM users ORDER BY id").df()
display(users)

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708016331.9650567,ee+2bOddftRt1A
1,2,Person_2,27,City_A,1708016331.9650567,buB03mQ6bN6Ilg
2,3,Person_3,28,City_A,1708016331.9650567,7Lx13dfGdkGVjg
3,4,Person_4,29,City_A,1708016331.9650567,M9jG9gwG0shYnA
4,5,Person_5,30,City_A,1708016331.9650567,sLf5STbayK9m7w


### Second Iterator

In [17]:
info = pipeline.run(data2, table_name="users", write_disposition="merge", primary_key="ID")

In [18]:
users = conn.sql("SELECT * FROM users ORDER BY id").df()
display(users)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708016331.9650567,ee+2bOddftRt1A,None
1,2,Person_2,27,City_A,1708016331.9650567,buB03mQ6bN6Ilg,None
2,3,Person_3,33,City_B,1708016380.2415948,sR/2OQMs/gIpvg,Job_3
3,4,Person_4,34,City_B,1708016380.2415948,Kr8EyLe8UQ5rNA,Job_4
4,5,Person_5,35,City_B,1708016380.2415948,i6rm4xYBNiu8kw,Job_5
5,6,Person_6,36,City_B,1708016380.2415948,Lap9Uu12QMqUuA,Job_6
6,7,Person_7,37,City_B,1708016380.2415948,aPzTnZPQR6am0A,Job_7
7,8,Person_8,38,City_B,1708016380.2415948,bns+bdlxuCe+9Q,Job_8


In [19]:
age_sum_2 = conn.sql("SELECT SUM(age) FROM users AS total_age").df()
display(age_sum_2)

,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX